In [1]:
categories = ['alt.atheism']#, 'soc.religion.christian','comp.graphics', 'sci.med']
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Load the list of files matching those categories as follows:

In [3]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

KeyboardInterrupt: 

In [ ]:
# Did we get what we wanted?
twenty_train.target_names

In [ ]:
# Size
len(twenty_train.data)

In [ ]:
# Lets see what we have :
print("\n".join(twenty_train.data[0].split("\n")[:2]))

In [ ]:
print(twenty_train.target_names[twenty_train.target[0]])

In [ ]:
print("\n".join(twenty_train.data[1000].split("\n")[:2]))

In [ ]:
print(twenty_train.target_names[twenty_train.target[1000]])

In [ ]:
#scikit-learn provides basic tools to process text using the Bag of Words representation.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Punctuation and single letter words will be automatically removed.

In [ ]:
count_vect = CountVectorizer(stop_words='english')

In [ ]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [ ]:
X_train_counts.shape

In [ ]:
#Occurrence count is a good start but there is an issue: 
#longer documents will have higher average count values than shorter documents, 
#even though they might talk about the same topics.

#To avoid these potential discrepancies it suffices to divide the number 
#of occurrences of each word in a document by the total number of words in the document: 
#these new features are called “tf” for Term Frequencies.

#Another refinement on top of tf is to downscale weights for words 
#that occur in many documents in the corpus and are therefore less 
#informative than those that occur only in a smaller portion of the corpus.

#This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer()

In [ ]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
# Import the SKlearn model we are using
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                      hidden_layer_sizes=(10, 10,10), random_state=1)
clf.fit(X_train_tfidf, twenty_train.target)

In [ ]:
docs_new = ['Doctors are bad', 'OpenGL on the GPU is fast']

In [ ]:
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [ ]:
predicted = clf.predict(X_new_tfidf)

In [ ]:
for doc, category in zip(docs_new, predicted):
   print('%r => %s' % (doc, twenty_train.target_names[category]))

In [ ]:
twenty_test = fetch_20newsgroups(
     subset='test', categories=categories,
     shuffle=True, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10,10), random_state=1)),
         ])

In [ ]:
text_clf.fit(twenty_train.data, twenty_train.target)

In [ ]:
predicted = text_clf.predict(twenty_test.data)

In [ ]:
import numpy as np
np.mean(predicted == twenty_test.target) 